In [10]:
#셀레니움 사용을 위한 임포트
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
#웹 드라이버 사용을 위한 임포트
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#sleep 사용을 위한 임포트
import time
#mysql 사용을 위한 임포트
import pymysql

#db 연결
conn = pymysql.connect(host='localhost', user='root', password='1234', db='toon_plus', charset='utf8')
#db 커서 설정
curs = conn.cursor()
#insert문 사전 정의
sql = "insert into toon_table (toon_num, N_toon, Toon_imagelink, Toon_link, Toon_category, Toon_update) values (%s, %s, %s, %s, %s, %s)"

#테이블 비우기
curs.execute("delete from toon_table")
conn.commit()

#크롤링 페이지 설정
option = Options()
option.add_argument("disable-infobars")
option.add_argument("disable-extensions")
# option.add_argument("start-maximized")
#option.add_argument('disable-gpu')
#option.add_argument('headless')
s = Service()
#웹툰 정보 수집을 위한 드라이버
browser = webdriver.Chrome(service=s, options=option)
wait = WebDriverWait(browser, 10)
#웹툰 정보 수집을 위한 페이지 열기
browser.get("https://webtoon.kakao.com/original-webtoon?tab=mon")
browser.implicitly_wait(10)
#바디 불러오기
actions = browser.find_element(By.CSS_SELECTOR, 'body')

#스크롤 내리는 부분
for j in range(45):
    time.sleep(1)
    actions.send_keys(Keys.PAGE_DOWN)

#웹툰 태그 수집을 위한 드라이버
specific_browser = webdriver.Chrome(service=s, options=option)
wait = WebDriverWait(specific_browser, 10)

#상단 노출 웹툰을 피하기 위한 정제
webtoon_container = browser.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[1]/a/div/div')
#웹툰 이름 수집
webtoon_name = webtoon_container.find_elements(By.CSS_SELECTOR, "img.w-full max-w-[115px] lg:max-w-[150px] mx-auto my-0")
#웹툰 이미지 수집
webtoon_img = webtoon_container.find_elements(By.CSS_SELECTOR, "img.w-full h-full object-cover object-top")
#웹툰 링크 수집
webtoon_link = webtoon_container.find_elements(By.CLASS_NAME, "w-full h-full relative overflow-hidden rounded-8 before:acsolute before:inset-0 before:bg-grey-01 before:-z-1")
#웹툰 업데이트 여부를 확인하기 위한 정제
webtoon_specific = webtoon_container.find_elements(By.CLASS_NAME, "whitespace-pre-wrap break-all break-words support-break-word font-badge  !whitespace-nowrap rounded-5 s10-bold-white bg-black px-4")

webtoon_name_for_db = []
webtoon_img_for_db = []
webtoon_link_for_db = []
    
sql_i = 0
tag_i = 0
update_i = 0

for element in webtoon_name:
    webtoon_name_for_db.append(str(element.text))

for element in webtoon_img:
    webtoon_img_for_db.append(str(element.get_attribute('src')))
    
webtoon_update = [0 for i in range(len(webtoon_img_for_db))]

for element in webtoon_specific:
    webtoon_update_check = element.find_element(By.CLASS_NAME, "w-full h-full relative overflow-hidden rounded-8 before:acsolute before:inset-0 before:bg-grey-01 before:-z-1")   
    tmp_webtoon_update = webtoon_update_check.find_elements(By.CLASS_NAME, "whitespace-pre-wrap break-all break-words support-break-word font-badge !whitespace-nowrap rounded-5 s10-bold-white bg-dark-grey-09 px-4")
    if None != tmp_webtoon_update:
        for i in tmp_webtoon_update:
            if i.text == "UP":
                webtoon_update[update_i] = 1
            else:
                print(i.text)
    else:
        webtoon_update[update_i] = 0
        
    update_i = update_i + 1
    
webtoon_tag_for_db = [" " for i in range(len(webtoon_img_for_db)) ]
    
for element in webtoon_link:
    webtoon_link_for_db.append(str(element.get_attribute('href')))
    specific_browser.get(str(element.get_attribute('href')))
    specific_browser.implicitly_wait(4)
    webtoon_tag = specific_browser.find_elements(By.CLASS_NAME, "TagGroup__tag--xu0OH")
    for tag in webtoon_tag:
        webtoon_tag_for_db[tag_i] = webtoon_tag_for_db[tag_i] + tag.text + " "
    time.sleep(2)
    tag_i = tag_i + 1
    
for idx in webtoon_link_for_db:
    curs.execute(sql, (sql_i, webtoon_name_for_db[sql_i], webtoon_img_for_db[sql_i], webtoon_link_for_db[sql_i], webtoon_tag_for_db[sql_i], webtoon_update[sql_i]))
    sql_i = sql_i + 1
conn.commit()
conn.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div[2]/div[1]/div[1]/a/div/div"}
  (Session info: chrome=116.0.5845.141); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66EDF52A2+57122]
	(No symbol) [0x00007FF66ED6EA92]
	(No symbol) [0x00007FF66EC3E3AB]
	(No symbol) [0x00007FF66EC77D3E]
	(No symbol) [0x00007FF66EC77E2C]
	(No symbol) [0x00007FF66ECB0B67]
	(No symbol) [0x00007FF66EC9701F]
	(No symbol) [0x00007FF66ECAEB82]
	(No symbol) [0x00007FF66EC96DB3]
	(No symbol) [0x00007FF66EC6D2B1]
	(No symbol) [0x00007FF66EC6E494]
	GetHandleVerifier [0x00007FF66F09EF82+2849794]
	GetHandleVerifier [0x00007FF66F0F1D24+3189156]
	GetHandleVerifier [0x00007FF66F0EACAF+3160367]
	GetHandleVerifier [0x00007FF66EE86D06+653702]
	(No symbol) [0x00007FF66ED7A208]
	(No symbol) [0x00007FF66ED762C4]
	(N